In [1]:
import numpy as np
import scipy.stats as st
import netCDF4 as nc
import datetime
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import csv
import multiprocessing
from percentile_fun import compute_percentile
import matplotlib.cm as cm
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from rain_get import rain_fill

In [2]:
def gen_time_series(arr, loc_num):
    """generate data across time, by interpreting the 3rd dim as time"""
    # [time,lon,lat]:[1748,1440,400] -> [57600,1748]:[data_plot,time]
    out = []
    for n in range(0,loc_num):
        for l in arr[:, :, n].T:
            out.append(l)
    out = np.array(out)
    return out

In [3]:
"""def data_clean(arr, chunk_size):
    num_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=num_processes)
    results = []
    for i in range(0, arr.shape[0], chunk_size):
        chunk = arr[i:i+chunk_size]
        result = pool.apply_async(rain_fill, (chunk,))
        results.append(result)
    pool.close()
    pool.join()
    return results

if __name__ == '__main__':
    percen_rain = data_clean(time_series[:288000], 72000)"""

"def data_clean(arr, chunk_size):\n    num_processes = multiprocessing.cpu_count()\n    pool = multiprocessing.Pool(processes=num_processes)\n    results = []\n    for i in range(0, arr.shape[0], chunk_size):\n        chunk = arr[i:i+chunk_size]\n        result = pool.apply_async(rain_fill, (chunk,))\n        results.append(result)\n    pool.close()\n    pool.join()\n    return results\n\nif __name__ == '__main__':\n    percen_rain = data_clean(time_series[:288000], 72000)"

In [6]:
# read lon and lat from abitary dates
test_set = nc.Dataset('data/20011224.nc4')
lat = test_set['lat'][:]
lon = test_set['lon'][:]

In [8]:
# read precipitation data from all the dates between 1998-2016
# JJA = 92
pcp_whole = []
dates_whole = []
years = [n for n in range(1998,2017)]
for y in years:
    start = datetime.date(y,6,1)
    for n in range(1,93):
        #change this according to the position
        str = 'data/' +  start.strftime("%Y%m%d") + '.nc4'
        nc_da = nc.Dataset(str)
        pcp_whole.append(nc_da['precipitation'][:])
        dates_whole.append(start.strftime("%Y%m%d"))
        start = start + datetime.timedelta(days=1)
    print(len(pcp_whole))
    print(pcp_whole[0].shape)
pcp_whole = np.array(pcp_whole)

92
(1440, 400)
184
(1440, 400)
276
(1440, 400)
368
(1440, 400)
460
(1440, 400)
552
(1440, 400)
644
(1440, 400)
736
(1440, 400)
828
(1440, 400)
920
(1440, 400)
1012
(1440, 400)
1104
(1440, 400)
1196
(1440, 400)
1288
(1440, 400)
1380
(1440, 400)
1472
(1440, 400)
1564
(1440, 400)
1656
(1440, 400)
1748
(1440, 400)


In [10]:
# rainning days are defined as the days with >= 1 precipitation 
day_rain = np.where(pcp_whole>=1,pcp_whole, np.nan)

In [12]:
# this will take some time
# what it does is explained in the function
time_series = gen_time_series(day_rain,400)
time_series.shape

(576000, 1748)

In [13]:
# use multi processing to compute the percentiles
def parallel_percentile(arr, chunk_size):
    num_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=num_processes)

    results = []
    for i in range(0, arr.shape[0], chunk_size):
        chunk = arr[i:i+chunk_size]
        result = pool.apply_async(compute_percentile, (chunk,))
        results.append(result)

    pool.close()
    pool.join()

    percentiles = np.concatenate([result.get() for result in results])

    return percentiles
if __name__ == '__main__':
    percen = parallel_percentile(time_series, 72000)

/Users/eliashanlm/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1395: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/eliashanlm/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1395: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/eliashanlm/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1395: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,
/Users/eliashanlm/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1395: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [15]:
percen.shape

(576000,)

In [16]:
# save the precentiles
np.savetxt('percentile_rain.txt', percen)